In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/train.csv')
df.head()

/var/folders/c3/96l18xtx7fz9rgy3dxg2crdm0000gn/T/ipykernel_16236/3245565085.py:1: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/train.csv')


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,20025,833,642,51,2,1,1,1,1,9.0,...,3.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,922731975
1,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923172525
2,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923833350
3,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,925195050
4,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,926576775


In [3]:
df = df.reindex(sorted(df.columns), axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830101 entries, 0 to 2830100
Data columns (total 55 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   circuitId               int64  
 1   company                 object 
 2   constructorId           int64  
 3   constructorRef          object 
 4   date                    object 
 5   dob                     object 
 6   driverId                int64  
 7   driverRef               object 
 8   driverStandingsId       int64  
 9   driver_code             object 
 10  driver_num              object 
 11  fastestLap              object 
 12  fastestLapTime          object 
 13  forename                object 
 14  fp1_date                object 
 15  fp1_time                object 
 16  fp2_date                object 
 17  fp2_time                object 
 18  fp3_date                object 
 19  fp3_time                object 
 20  grand_prix              object 
 21  grid                    int64  

In [5]:
df.shape

(2830101, 55)

## Preprocessing

In [6]:
# Drop Irrelevant Columns
df.drop(columns={'driverId', 'circuitId', 'raceId_y', 'resultId', 'driverStandingsId', 'constructorId', 'racerId', 'result_driver_standing'}, inplace=True) #drop foreign keys that were used to merge, see f1db_user_guide

df.drop(columns={'wins', 'points_y', 'positionText_x', 'positionText_y', 'position'}, inplace=True) #duplicate carried over from driver standings table

df.drop(columns={'url', 'url_x', 'url_y'}, inplace=True) # links from tables

# Renaming Columns
df.rename(columns={'nationality_y': 'nationality_constructor'}, inplace=True)
df.rename(columns={'points': 'points_race'}, inplace=True) #points earned in race
df.rename(columns={'position_x': 'position'}, inplace=True) #target variable
#df.rename(columns={'points_y': 'points_driver'}, inplace=True) #drivers overall season points

# Handle NaN

df.replace('\\N', np.nan, inplace=True)


/var/folders/c3/96l18xtx7fz9rgy3dxg2crdm0000gn/T/ipykernel_16236/3789560837.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('\\N', np.nan, inplace=True)


In [7]:
preprocessed = df.drop_duplicates()
preprocessed.shape

(22098, 39)

In [8]:
df_hamilton = preprocessed[df['driver_code'] == 'HAM']
df_hamilton.to_csv('../data/df_hamilton.csv', index=False)
df_hamilton

/var/folders/c3/96l18xtx7fz9rgy3dxg2crdm0000gn/T/ipykernel_16236/111141921.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hamilton = preprocessed[df['driver_code'] == 'HAM']


(128, 39)

## Handle Datatypes

In [9]:
preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22098 entries, 0 to 2829963
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company                  22098 non-null  object 
 1   constructorRef           22098 non-null  object 
 2   date                     22098 non-null  object 
 3   dob                      22098 non-null  object 
 4   driverRef                22098 non-null  object 
 5   driver_code              5536 non-null   object 
 6   driver_num               1864 non-null   object 
 7   fastestLap               3827 non-null   object 
 8   fastestLapTime           3827 non-null   object 
 9   forename                 22098 non-null  object 
 10  fp1_date                 0 non-null      float64
 11  fp1_time                 0 non-null      float64
 12  fp2_date                 0 non-null      float64
 13  fp2_time                 0 non-null      float64
 14  fp3_date                 

In [18]:
# Define the data types for each column in the simulated Ergast database DataFrame
column_dtypes = {
    'company': 'string',
    'constructorRef': 'string',
    'date': 'datetime64[ns]',
    'dob': 'datetime64[ns]',
    'driverRef': 'string',
    'driver_code': 'string',
    'driver_num': 'Int64',
    'fastestLap': 'Int64',
    'fastestLapTime': 'string',
    'forename': 'string',
    'fp1_date': 'datetime64[ns]',
    'fp2_date': 'datetime64[ns]',
    'fp3_date': 'datetime64[ns]',
    'grand_prix': 'string',
    'grid': 'int64',
    'laps': 'int64',
    'max_speed': 'float64',
    'nationality': 'string',
    'nationality_constructor': 'string',
    'number': 'Int64',
    'points_race': 'float64',
    'positionOrder': 'int64',
    'position': 'string',
    'quali_date': 'datetime64[ns]',
    'quali_time': 'string',
    'rank': 'Int64',
    'round': 'int64',
    'sprint_date': 'datetime64[ns]',
    'status': 'string',
    'statusId': 'int64',
    'surname': 'string',
    'time_x': 'string',
    'time_y': 'string',
    'timetaken_in_millisec': 'Int64',
    #'url': 'string',
    #'url_x': 'string',
    #'url_y': 'string',
    'year': 'int64'
}

# Columns to be converted to timedelta
timedelta_columns = ['fp1_time', 'fp2_time', 'fp3_time', 'sprint_time']

# Ensure preprocessed is a deep copy if it was derived from another DataFrame
preprocessed = preprocessed.copy(deep=True)

# Convert the relevant columns to timedelta using .loc[]
for col in timedelta_columns:
    preprocessed.loc[:, col] = pd.to_timedelta(preprocessed[col], errors='coerce')

# Clean non-numeric entries in integer columns by coercing errors to NaN
for col, dtype in column_dtypes.items():
    if dtype in ['int64', 'Int64']:
        preprocessed.loc[:, col] = pd.to_numeric(preprocessed[col], errors='coerce')

# Now apply the column type conversions
preprocessed = preprocessed.astype(column_dtypes)

# Display the data types to confirm
print(preprocessed.dtypes)


company                     string[python]
constructorRef              string[python]
date                        datetime64[ns]
dob                         datetime64[ns]
driverRef                   string[python]
driver_code                 string[python]
driver_num                           Int64
fastestLap                           Int64
fastestLapTime              string[python]
forename                    string[python]
fp1_date                    datetime64[ns]
fp1_time                   timedelta64[ns]
fp2_date                    datetime64[ns]
fp2_time                   timedelta64[ns]
fp3_date                    datetime64[ns]
fp3_time                   timedelta64[ns]
grand_prix                  string[python]
grid                                 int64
laps                                 int64
max_speed                          float64
nationality                 string[python]
nationality_constructor     string[python]
number                               Int64
points_race

In [23]:
# Convert "MM:SS.SSS" format directly to timedelta using pd.to_timedelta()
preprocessed['fp1_timedelta'] = pd.to_timedelta(preprocessed['fp1_time'].fillna('0:00.000').apply(lambda x: f"0:{x}"))
preprocessed['fp2_timedelta'] = pd.to_timedelta(preprocessed['fp2_time'].fillna('0:00.000').apply(lambda x: f"0:{x}"))
preprocessed['fp3_timedelta'] = pd.to_timedelta(preprocessed['fp3_time'].fillna('0:00.000').apply(lambda x: f"0:{x}"))

# Convert timedelta to total seconds for ML purposes
preprocessed['fp1_time_sec'] = preprocessed['fp1_timedelta'].dt.total_seconds()
preprocessed['fp2_time_sec'] = preprocessed['fp2_timedelta'].dt.total_seconds()
preprocessed['fp3_time_sec'] = preprocessed['fp3_timedelta'].dt.total_seconds()

# Drop original time columns and timedelta columns if only seconds are needed for ML
preprocessed = preprocessed.drop(columns=['fp1_time', 'fp2_time', 'fp3_time', 'fp1_timedelta', 'fp2_timedelta', 'fp3_timedelta'])

# Display the resulting DataFrame
preprocessed

,company,constructorRef,date,dob,driverRef,driver_code,driver_num,fastestLap,fastestLapTime,forename,...,status,statusId,surname,time_x,time_y,timetaken_in_millisec,year,fp1_time_sec,fp2_time_sec,fp3_time_sec
0,Alfa Romeo,alfa,1950-05-13,1906-10-30,farina,<NA>,<NA>,<NA>,<NA>,Nino,...,Finished,1,Farina,2:13:23.6,<NA>,8003600,1950,0.0,0.0,0.0
48,Alfa Romeo,alfa,1950-05-13,1898-06-09,fagioli,<NA>,<NA>,<NA>,<NA>,Luigi,...,Finished,1,Fagioli,+2.6,<NA>,8006200,1950,0.0,0.0,0.0
60,Alfa Romeo,alfa,1950-05-13,1911-07-02,reg_parnell,<NA>,<NA>,<NA>,<NA>,Reg,...,Finished,1,Parnell,+52.0,<NA>,8055600,1950,0.0,0.0,0.0
81,Talbot-Lago,lago,1950-05-13,1904-10-08,cabantous,<NA>,<NA>,<NA>,<NA>,Yves,...,+2 Laps,12,Cabantous,<NA>,<NA>,<NA>,1950,0.0,0.0,0.0
106,Talbot-Lago,lago,1950-05-13,1905-11-05,rosier,<NA>,<NA>,<NA>,<NA>,Louis,...,+2 Laps,12,Rosier,<NA>,<NA>,<NA>,1950,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829461,Caterham,caterham,2013-11-17,1990-02-15,pic,PIC,<NA>,48,1:43.968,Charles,...,+1 Lap,11,Pic,<NA>,19:00:00,<NA>,2013,0.0,0.0,0.0
2829500,Marussia,marussia,2013-11-17,1991-04-21,chilton,CHI,4,48,1:43.775,Max,...,+2 Laps,12,Chilton,<NA>,19:00:00,<NA>,2013,0.0,0.0,0.0
2829538,Force India,force_india,2013-11-17,1983-01-11,sutil,SUT,99,<NA>,<NA>,Adrian,...,Accident,3,Sutil,<NA>,19:00:00,<NA>,2013,0.0,0.0,0.0
2829664,Red Bull,red_bull,2013-11-24,1987-07-03,vettel,VET,5,51,1:15.624,Sebastian,...,Finished,1,Vettel,1:32:36.300,16:00:00,5556300,2013,0.0,0.0,0.0


Calculate Driver Age based on DOB and Date

In [24]:
# Convert 'date' and 'dob' to datetime format
preprocessed['date'] = pd.to_datetime(preprocessed['date'])
preprocessed['dob'] = pd.to_datetime(preprocessed['dob'])

# Calculate age based on 'date' and 'dob'
preprocessed['age'] = preprocessed['date'].dt.year - preprocessed['dob'].dt.year - (
    (preprocessed['date'].dt.month < preprocessed['dob'].dt.month) | 
    ((preprocessed['date'].dt.month == preprocessed['dob'].dt.month) & (preprocessed['date'].dt.day < preprocessed['dob'].dt.day))
).astype(int)

# Drop the original 'date' and 'dob' columns
preprocessed = preprocessed.drop(columns=['date', 'dob'], kee)

# Display the resulting DataFrame
preprocessed

,company,constructorRef,driverRef,driver_code,driver_num,fastestLap,fastestLapTime,forename,fp1_date,fp2_date,...,statusId,surname,time_x,time_y,timetaken_in_millisec,year,fp1_time_sec,fp2_time_sec,fp3_time_sec,age
0,Alfa Romeo,alfa,farina,<NA>,<NA>,<NA>,<NA>,Nino,NaT,NaT,...,1,Farina,2:13:23.6,<NA>,8003600,1950,0.0,0.0,0.0,43
48,Alfa Romeo,alfa,fagioli,<NA>,<NA>,<NA>,<NA>,Luigi,NaT,NaT,...,1,Fagioli,+2.6,<NA>,8006200,1950,0.0,0.0,0.0,51
60,Alfa Romeo,alfa,reg_parnell,<NA>,<NA>,<NA>,<NA>,Reg,NaT,NaT,...,1,Parnell,+52.0,<NA>,8055600,1950,0.0,0.0,0.0,38
81,Talbot-Lago,lago,cabantous,<NA>,<NA>,<NA>,<NA>,Yves,NaT,NaT,...,12,Cabantous,<NA>,<NA>,<NA>,1950,0.0,0.0,0.0,45
106,Talbot-Lago,lago,rosier,<NA>,<NA>,<NA>,<NA>,Louis,NaT,NaT,...,12,Rosier,<NA>,<NA>,<NA>,1950,0.0,0.0,0.0,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829461,Caterham,caterham,pic,PIC,<NA>,48,1:43.968,Charles,NaT,NaT,...,11,Pic,<NA>,19:00:00,<NA>,2013,0.0,0.0,0.0,23
2829500,Marussia,marussia,chilton,CHI,4,48,1:43.775,Max,NaT,NaT,...,12,Chilton,<NA>,19:00:00,<NA>,2013,0.0,0.0,0.0,22
2829538,Force India,force_india,sutil,SUT,99,<NA>,<NA>,Adrian,NaT,NaT,...,3,Sutil,<NA>,19:00:00,<NA>,2013,0.0,0.0,0.0,30
2829664,Red Bull,red_bull,vettel,VET,5,51,1:15.624,Sebastian,NaT,NaT,...,1,Vettel,1:32:36.300,16:00:00,5556300,2013,0.0,0.0,0.0,26


## Adjust for Team Aquisitions etc.

In [43]:
# Create the mapping dictionary
team_mapping = {
    'Jordan': 'Aston Martin',
    'Midland': 'Aston Martin',
    'MF1': 'Aston Martin',
    'Spyker': 'Aston Martin',
    'Spyker MF1': 'Aston Martin',
    'Force India': 'Aston Martin',
    'Racing Point': 'Aston Martin',
    'Aston Martin': 'Aston Martin',

    'Tyrrell': 'Mercedes',
    'BAR': 'Mercedes',
    'Honda': 'Mercedes',
    'Brawn': 'Mercedes',
    'Brawn GP': 'Mercedes',
    'Mercedes': 'Mercedes',

    'Toleman': 'Alpine',
    'Benetton': 'Alpine',
    'Renault': 'Alpine',
    'Lotus F1': 'Alpine',
    'Lotus': 'Alpine',  # Only if referring to the Lotus F1 Team from 2012-2015
    'Alpine': 'Alpine',

    'Minardi': 'AlphaTauri',
    'Toro Rosso': 'AlphaTauri',
    'AlphaTauri': 'AlphaTauri',

    'Stewart': 'Red Bull Racing',
    'Jaguar': 'Red Bull Racing',
    'Red Bull': 'Red Bull Racing',
    'Red Bull Racing': 'Red Bull Racing',

    'Sauber': 'Alfa Romeo',
    'BMW Sauber': 'Alfa Romeo',
    'Alfa Romeo': 'Alfa Romeo',  # Referring to the current Sauber-run team

    'McLaren': 'McLaren',
    'McLaren-Ford': 'McLaren',
    'McLaren-Serenissima': 'McLaren',
    'McLaren-BRM': 'McLaren',
    'McLaren-Alfa Romeo': 'McLaren',

    'Williams': 'Williams',

    'Ferrari': 'Ferrari',
}

# Assuming your DataFrame is named df and the team names are in a column called 'Team'
df['company'] = df['company'].replace(team_mapping)


## Save Preprocessed Data

In [44]:

preprocessed.to_csv('../data/preprocessed.csv', index=False)
